# QVERIFY: Specification Synthesis Deep Dive

This notebook explores the specification synthesis capabilities of QVERIFY in detail.

## Topics Covered
1. QuantumSpecSynth algorithm internals
2. Quantum-aware prompting strategies
3. Counterexample-guided refinement (CEGIS)
4. Loop invariant synthesis
5. Customizing synthesis behavior

In [ ]:
from qverify import QVerify, QuantumProgram
from qverify.algorithms import QuantumSpecSynth, LearnQuantumPredicate, SynthesizeQuantumInvariant
from qverify.core.specification import Specification, Precondition, Postcondition

## 1. Understanding QuantumSpecSynth

The QuantumSpecSynth algorithm uses LLMs with quantum-aware prompting to generate specifications.

In [ ]:
# Initialize the synthesizer directly
from qverify.utils.llm_interface import create_llm_interface, LLMConfig

# Create LLM interface
llm = create_llm_interface(LLMConfig(
    provider="anthropic",
    model="claude-3.5-sonnet",
    temperature=0.2
))

# Create synthesizer with custom settings
synth = QuantumSpecSynth(
    llm=llm,
    max_candidates=5,
    max_refinement_iterations=3,
    enable_quantum_aware_prompting=True
)

print("QuantumSpecSynth initialized")

## 2. Program Analysis

Before synthesis, the algorithm analyzes the quantum program structure:

In [ ]:
# Load a program
grover_code = """
def grover_iteration(qubits: qubit[]) {
    qubits = hadamard_all(qubits);
    qubits = oracle(qubits);
    qubits = diffusion(qubits);
    return qubits;
}
"""
program = QuantumProgram.from_silq(grover_code, name="grover")

# Analyze the program
analysis = synth._analyze_program(program)

print("Program Analysis:")
print(f"  Qubits: {analysis.num_qubits}")
print(f"  Has loops: {analysis.has_loops}")
print(f"  Has measurements: {analysis.has_measurements}")
print(f"  Gate sequence: {analysis.gate_sequence}")

## 3. Counterexample-Guided Refinement

When initial specifications fail verification, QVERIFY uses counterexamples to refine them:

In [ ]:
from qverify.algorithms import RepairSpecification
from qverify.core.types import CounterExample, QuantumState
import numpy as np

# Create a mock counterexample
cex = CounterExample(
    input_state=QuantumState(
        num_qubits=2,
        amplitudes=np.array([1, 0, 0, 0], dtype=complex)
    ),
    output_state=QuantumState(
        num_qubits=2,
        amplitudes=np.array([0.5, 0.5, 0.5, 0.5], dtype=complex)
    ),
    violated_condition="postcondition"
)

# Initialize repairer
repairer = RepairSpecification(llm=llm)

print("Counterexample created for demonstration")

## 4. Loop Invariant Synthesis

For iterative quantum algorithms, QVERIFY can synthesize loop invariants:

In [ ]:
# Initialize invariant synthesizer
inv_synth = SynthesizeQuantumInvariant(llm=llm, max_unrolling=5)

print("Loop invariant synthesizer ready")
print("\nFor Grover's algorithm, typical invariants include:")
print("  - prob(marked) increases with each iteration")
print("  - amplitude of marked state ≈ sin((2k+1)θ) after k iterations")
print("  - state remains normalized")

## 5. Custom Synthesis Configuration

In [ ]:
# Configure synthesis for different scenarios

# High accuracy (more candidates, more refinement)
high_accuracy_synth = QuantumSpecSynth(
    llm=llm,
    max_candidates=10,
    max_refinement_iterations=5,
    timeout_seconds=120.0
)

# Fast (fewer candidates, less refinement)
fast_synth = QuantumSpecSynth(
    llm=llm,
    max_candidates=2,
    max_refinement_iterations=1,
    timeout_seconds=30.0
)

print("Created high-accuracy and fast synthesizer configurations")

## Next Steps

- See `03_verification_deep_dive.ipynb` for verification details
- Explore the `algorithms/` source code for implementation details